In [ ]:
import numpy as np
import time

# 1. Fonction Python standard (ne gère que des scalaires ou listes simples)
def ma_fonction_vectorisee(x, y):
    """Calcule la distance euclidienne entre x et y"""
    return np.sqrt(x**2 + y**2)

# 2. Conversion en UFunc (Universal Function)
# 2 entrées (x, y), 1 sortie
ufunc_perso = np.frompyfunc(ma_fonction_vectorisee, 2, 1)

# Test sur de petits tableaux
x_test = np.array([1, 2, 3, 4])
y_test = np.array([5, 6, 7, 8])

resultat_test = ufunc_perso(x_test, y_test)

print("Test sur petits tableaux :")
print(f"x : {x_test}")
print(f"y : {y_test}")
print(f"Résultat : {resultat_test}")
# Notez que le type est souvent 'object' avec frompyfunc
print(f"Type résultat : {type(resultat_test)}")

In [ ]:
def test_performance_grand_dataset():
    taille = 1_000_000
    x = np.arange(taille, dtype=np.float64)
    y = np.arange(taille, dtype=np.float64)

    print(f"--- Test de performance sur {taille:,} éléments ---")

    # Méthode 1: UFunc personnalisée
    start = time.time()
    resultat_ufunc = ufunc_perso(x, y)
    temps_ufunc = time.time() - start

    # Méthode 2: Boucle Python (Très lent)
    start = time.time()
    resultat_boucle = np.empty(taille)
    for i in range(taille):
        resultat_boucle[i] = ma_fonction_vectorisee(x[i], y[i])
    temps_boucle = time.time() - start

    # Méthode 3: Vectorisation NumPy native (Le standard à viser)
    start = time.time()
    resultat_native = np.sqrt(x**2 + y**2)
    temps_native = time.time() - start

    # Affichage des résultats
    print(f"UFunc personnalisée : {temps_ufunc:.4f} s")
    print(f"Boucle Python       : {temps_boucle:.4f} s")
    print(f"NumPy natif         : {temps_native:.4f} s")
    
    if temps_ufunc > 0:
        print(f"Accélération Boucle vs UFunc : {temps_boucle / temps_ufunc:.1f}x")
    
    # Vérification de la précision
    # Attention : frompyfunc renvoie des objets, on convertit en float pour comparer
    res_ufunc_float = resultat_ufunc.astype(float)
    est_identique = np.allclose(res_ufunc_float, resultat_native)
    print(f"\nPrécision UFunc vs Native : {est_identique}")

test_performance_grand_dataset()

In [ ]:
# UFunc avec gestion des types et broadcasting
def fonction_complexe(x, y, parametre=1.0):
    """
    Fonction mathématique complexe avec paramètre
    Inclut la gestion des cas particuliers
    """
    # Gestion des valeurs négatives
    if x < 0 or y < 0:
        return np.nan
        
    # Calcul
    return np.sin(x * y) * parametre + np.cos(x - y)

# Création d'UFunc avec signature de types
# otypes=['float'] force le résultat en float (plus rapide que object)
# excluded=['parametre'] empêche numpy de vectoriser cet argument
ufunc_avancee = np.vectorize(fonction_complexe, 
                             otypes=[np.float64], 
                             excluded=['parametre'])

# Test avec broadcasting (Meshgrid)
x = np.linspace(0, 2*np.pi, 5)
y = np.linspace(0, np.pi, 3)

# Création de la grille 2D
X, Y = np.meshgrid(x, y)

print("Matrice X (shape):", X.shape)
print("Matrice Y (shape):", Y.shape)

# Application de la fonction sur toute la matrice d'un coup
resultat_avance = ufunc_avancee(X, Y, parametre=2.0)

print("\nRésultat avec broadcasting :")
print(resultat_avance)